In [1]:
from sklearn.ensemble import GradientBoostingClassifier as GB
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.model_selection import train_test_split as split
from ucimlrepo import fetch_ucirepo
import numpy as np
import pandas as pd

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import BallTree
from sklearn.preprocessing import StandardScaler
from psyke import Extractor
from psyke.utils.logic import pretty_theory
from psyke.extraction.hypercubic import Grid, FeatureRanker
from psyke.extraction.hypercubic.strategy import AdaptiveStrategy

In [2]:
# fetch dataset 
adult = fetch_ucirepo(id=2)
X = adult.data.features[['age', 'fnlwgt', 'education-num', 'race', 'sex',
                         'capital-gain', 'capital-loss', 'hours-per-week']]
y = adult.data.targets

l = list(np.unique(X.race))
y.income = y.income.apply(lambda x: x[:-1] if x[-1] == '.' else x)
X.sex = X.sex.apply(lambda x: 0 if x == 'Male' else 1)
X.race = X.race.apply(lambda x: l.index(x))

scaler = StandardScaler()
scaler.fit(X)
X = pd.DataFrame(scaler.transform(X), index=X.index, columns=X.columns)

normalization = {key: (m, s) for key, m, s in zip(X.columns, scaler.mean_, scaler.scale_)}

data = X.join(y)

In [3]:
def remove(l):
    for i in l:
        if 'sex' in i:
            del i['sex']
        if 'race' in i:
            del i['race']

In [4]:
cols = ['accuracy', 'fidelity', 'ind-bias', 'ind-treat',
        'group-bias-M', 'group-bias-F', 'group-treat-M', 'group-treat-F']

results = {
    'GB': {c: [] for c in cols},
    'KNN': {c: [] for c in cols},
    'SVM': {c: [] for c in cols}
}

for seed in [123, 345, 874, 128, 963, 741, 159, 853, 698, 742]:
    print(f"********************************************{seed}********************************************************")

    train, test = split(data, test_size=.5, random_state=seed)

    M = test['sex'] == min(test['sex'])
    F = test['sex'] == max(test['sex'])

    actionable_test = test.drop(['sex', 'race', 'income'], axis=1)
    M_test = test[test.sex == min(test.sex)].drop(['sex', 'race', 'income'], axis=1)
    F_test = test[test.sex == max(test.sex)].drop(['sex', 'race', 'income'], axis=1)
    closest = BallTree(actionable_test, p=2).query(actionable_test, k=2)[1][:, 1]
    closest_M = BallTree(M_test, p=2).query(F_test, k=2)[1][:, 1]
    closest_F = BallTree(F_test, p=2).query(M_test, k=2)[1][:, 1]

    gb = GB(n_estimators=200, learning_rate=0.2, random_state=seed).fit(train.iloc[:, :-1], train.iloc[:, -1])
    print(f"GB accuracy = {gb.score(test.iloc[:, :-1], test.iloc[:, -1]):.2f}")
    results['GB']['accuracy'].append(gb.score(test.iloc[:, :-1], test.iloc[:, -1]))

    knn = KNN(n_neighbors=150).fit(train.iloc[:, :-1], train.iloc[:, -1])
    print(f"KNN accuracy = {knn.score(test.iloc[:, :-1], test.iloc[:, -1]):.2f}")
    results['KNN']['accuracy'].append(knn.score(test.iloc[:, :-1], test.iloc[:, -1]))

    svc = SVC(C=0.85, random_state=seed).fit(train.iloc[:, :-1], train.iloc[:, -1])
    print(f"SVM accuracy = {svc.score(test.iloc[:, :-1], test.iloc[:, -1]):.2f}")
    results['SVM']['accuracy'].append(svc.score(test.iloc[:, :-1], test.iloc[:, -1]))

    for i, (name, predictor) in enumerate(zip(['GB', 'KNN', 'SVM'], [gb, knn, svc])):
        extractor = Extractor.gridex(predictor, Grid(1, AdaptiveStrategy(
            FeatureRanker(data.columns[:-1]).fit(predictor, data.iloc[:, :-1]).rankings(), [(0.15, 2), (0.9, 3)]
        )), threshold=.1, min_examples=100, normalization=normalization)
        theory = extractor.extract(train)
        fid = extractor.accuracy(test, predictor)
        print(f'\nExtractor performance ({extractor.n_rules} rules): Fidelity = {fid:.2f}')
        results[name]['fidelity'].append(fid)

        res = [extractor.predict_counter(dict(r[:-1]), verbose=False, only_first=True)[1][0][1]
               for _, r in test.iterrows()]
        res = [('sex' in r and r['sex'] != []) or ('race' in r and r['race'] != []) for r in res]

        results[name]['ind-bias'].append(sum(np.array(res)) / len(test))
        results[name]['group-bias-M'].append(sum(np.array(res)[M]) / sum(M))
        results[name]['group-bias-F'].append(sum(np.array(res)[F]) / sum(F))

        z1 = [extractor.predict_counter(dict(r[:-1]), verbose=False, only_first=True)[1][0][1]
              for j, (_, r) in enumerate(test.iterrows())]
        z2 = [extractor.predict_counter(dict(test.iloc[closest[j], :-1]), verbose=False, only_first=True)[1][0][1]
              for j, (_, r) in enumerate(test.iterrows())]

        zm1 = [extractor.predict_counter(dict(r[:-1]), verbose=False, only_first=True)[1][0][1]
               for j, (_, r) in enumerate(M_test.iterrows())]
        zm2 = [extractor.predict_counter(dict(F_test.iloc[closest_F[j], :-1]), verbose=False, only_first=True)[1][0][1]
               for j, (_, r) in enumerate(M_test.iterrows())]

        zf1 = [extractor.predict_counter(dict(r[:-1]), verbose=False, only_first=True)[1][0][1]
               for j, (_, r) in enumerate(F_test.iterrows())]
        zf2 = [extractor.predict_counter(dict(M_test.iloc[closest_M[j], :-1]), verbose=False, only_first=True)[1][0][1]
               for j, (_, r) in enumerate(F_test.iterrows())]

        for x in [z1, z2, zm1, zm2, zf1, zf2]:
            remove(x)

        results[name]['ind-treat'].append(
            sum([zz1 != zz2 for zz1, zz2 in zip(z1, z2)]) / len(test)
        )
        results[name]['group-treat-M'].append(
            sum([zz1 != zz2 for zz1, zz2 in zip(zm1, zm2)]) / len(M_test)
        )
        results[name]['group-treat-F'].append(
            sum([zz1 != zz2 for zz1, zz2 in zip(zf1, zf2)]) / len(F_test)
        )

********************************************123********************************************************
GB accuracy = 0.85
KNN accuracy = 0.82
SVM accuracy = 0.83

Extractor performance (26 rules): Fidelity = 0.87

Extractor performance (23 rules): Fidelity = 0.90

Extractor performance (21 rules): Fidelity = 0.89
********************************************345********************************************************
GB accuracy = 0.85
KNN accuracy = 0.82
SVM accuracy = 0.83

Extractor performance (25 rules): Fidelity = 0.87

Extractor performance (26 rules): Fidelity = 0.90

Extractor performance (21 rules): Fidelity = 0.88
********************************************874********************************************************
GB accuracy = 0.85
KNN accuracy = 0.82
SVM accuracy = 0.83

Extractor performance (17 rules): Fidelity = 0.88

Extractor performance (18 rules): Fidelity = 0.90

Extractor performance (20 rules): Fidelity = 0.88
********************************************128*****

In [7]:
for name in ['GB', 'KNN', 'SVM']:
    print(name, end=' & ')
    for k in cols[:2]:
        print(f"{np.mean(results[name][k]):.2f} $\\pm$ {np.std(results[name][k]):.2f}", end=' & ')
    print("\\\\")

GB & 0.85 $\pm$ 0.00 & 0.87 $\pm$ 0.00 & \\
KNN & 0.82 $\pm$ 0.00 & 0.90 $\pm$ 0.00 & \\
SVM & 0.83 $\pm$ 0.00 & 0.88 $\pm$ 0.00 & \\
